In [1]:
import pandas as pd
import os
import glob
import numpy as np
import ipyvuetify as v
import ipywidgets as widgets
from ipywidgets import Layout, Button, Box, VBox, HBox
from ipyleaflet import AwesomeIcon, Marker, Map, Icon, Popup, basemaps, Polyline
from bqplot import Axis, Figure, Lines, LinearScale
import bqplot as bq
import matplotlib.pyplot as plt
import matplotlib as mpl
import io
#from stl import mesh
#import ipyvolume as ipv

In [2]:
# TO DO : add title, image (inav, x-lite, moi qui vole sur l'aile)
# sort value before fitting ? it will destroy time slider interaction ?
# add HD video visualisation

In [3]:
widgets.HTML('<center> <img src="images/gaju_fpv.gif" align="middle" class="center"> </center>')

HTML(value='<center> <img src="images/gaju_fpv.gif" align="middle" class="center"> </center>')

In [4]:
def get_rssi(RSSI, dist):
    [a,b] = np.polyfit(np.log(dist), RSSI, 1)
    rssi_time = a * np.log(dist) + b
    rssi_dist = a * np.log(np.sort(dist)) + b
    return rssi_time, rssi_dist

def get_max(data, data_type='str'):
    max_dist  = np.max(data['Dist(m)'])
    max_speed = np.max(data['GSpd(kmh)'])
    max_alt   = np.max(data['Alt(m)'])
    if data_type=="str":
        max_dist  = str(max_dist)
        max_speed = str(max_speed)
        max_alt   = str(max_alt)
    return max_dist, max_speed, max_alt

In [5]:
files = glob.glob('logs/*.csv')

In [6]:
# Get max stats
max_dist_tot, max_speed_tot, max_alt_tot = 0, 0, 0
for file in files:
    data = pd.read_csv(file, sep=';')
    new_dist, new_speed, new_alt = get_max(data, data_type='int')
    max_dist_tot  = np.max([max_dist_tot,new_dist])
    max_speed_tot = np.max([max_speed_tot,new_speed])
    max_alt_tot   = np.max([max_alt_tot,new_alt])
max_dist_tot  = str(max_dist_tot)
max_speed_tot = str(max_speed_tot)
max_alt_tot   = str(max_alt_tot)

In [7]:
class TelemetryData:
    def __init__(self):
        self.dist, self.rssi = [], []
        self.lon, self.lat   = [], []
        self.time = []
        self.rssi_time, self.rssi_dist = [], []
        self.dist, self.speed, self.alt = 0, 0, 0
        self.max_dist, self.max_speed, self.max_alt = 0, 0, 0
        self.data = []
    def get_data(self, file):
        self.data = pd.read_csv(file, sep=';', parse_dates=[0,1] , infer_datetime_format=True)
        #self.data = data.interpolate(method='nearest', column=['GPS(lat)','GPS(lon)']).ffill().bfill()
        self.data = self.data[~(self.data['GPS(lat)'].isnull())]
        dist, rssi = self.data['Dist(m)'], self.data['RSSI(dB)']
        filt = (rssi != 0) & (dist != 0)
        self.dist = dist[filt]
        self.rssi = rssi[filt]
        self.lon  = self.data['GPS(lon)'][filt]
        self.lat  = self.data['GPS(lat)'][filt]
        self.time = np.arange(len(self.data['Time']))
        self.rssi_time, self.rssi_dist = get_rssi(self.rssi, self.dist)
        self.speed, self.alt = self.data['GSpd(kmh)'][filt], self.data['Alt(m)'][filt]
        self.max_dist, self.max_speed, self.max_alt = get_max(self.data)

In [8]:
# Initialize Data
tlm = TelemetryData()
tlm.get_data(files[0])

In [9]:
# MAIN COMPONENTS

In [10]:
out = widgets.Output()

In [11]:
# left - files
uploader    = widgets.FileUpload(accept='.csv', multiple=True, layout=Layout(width='14vw'))
file_select = widgets.Dropdown(options=[os.path.basename(f) for f in files], layout=Layout(width='14vw'))
#if file_exist:
#video       = widgets.Video.from_file("videos/Big.Buck.Bunny.mp4", autoplay=False,layout=Layout(width='14vw'))
ar_wing    = widgets.HTML('<center> <img src="images/ar_wing.png" align="middle" class="center"> </center>')
dist_curr  = widgets.HTML(value='<b>Distance (m)</b> : '+str(tlm.dist.iloc[100]))
speed_curr = widgets.HTML(value='<b>Speed (km/h)</b> : '+str(tlm.speed.iloc[100]))
alt_curr   = widgets.HTML(value='<b>Altitude (m)</b> : '+str(tlm.alt.iloc[100]))
infos      = VBox(children=[dist_curr, speed_curr, alt_curr], layout=Layout(align_items='center', border='solid 1px black', flex_direction='column', align_content='flex-end'))
infos_     = VBox(children=[ar_wing, infos], layout=Layout(align_items='center', flex_direction='column', align_content='flex-end'))
left       = VBox(children=[uploader,file_select,infos_])#,video])

# center - map and time slider
my_map      = Map(basemap=basemaps.Esri.WorldImagery, center=(np.mean(tlm.lat), np.mean(tlm.lon)), zoom=16)
icon_home   = AwesomeIcon(name='home', icon_color="white", marker_color='black')
marker_home = Marker(icon=icon_home, location=(tlm.lat.iloc[0], tlm.lon.iloc[0]))
line_map    = Polyline(locations=[(x, y) for x, y in zip(tlm.lat, tlm.lon)], color = "red", fill=False, weight=2)
time_slider = widgets.IntSlider(value=len(tlm.time), max=len(tlm.time), layout=Layout(flex='1.5 1 0%', width='auto'))
play_button = widgets.Play(max=len(tlm.time))
animation   = widgets.HBox([play_button, time_slider]) 
my_map.add_layer(marker_home)
my_map.add_layer(line_map)
widgets.jslink((play_button, 'value'), (time_slider, 'value'))
center = VBox(children=[my_map, animation], layout=Layout(flex='1.5 1 0%', width='auto', height='80vh', align_content='center', justify_content="center"))
    
# right - rssi graphs and records
x_var = widgets.Dropdown(options=tlm.data.columns, description='X axis :', value = 'Dist(m)', layout=Layout(width='12vw'))
y_var = widgets.Dropdown(options=tlm.data.columns, description='Y axis :', value = 'RSSI(dB)', layout=Layout(width='12vw'))
fit_button = widgets.ToggleButton(value=False, description='Fit', button_style='', icon='bolt', layout=Layout(width='5vw'))
xy_var  = widgets.HBox([x_var, y_var, fit_button], layout=Layout(width='auto', height='auto', align_content='center', justify_content="center")) 
xs, ys  = bq.LinearScale(), bq.LinearScale()
y_axis  = bq.Axis(scale=ys, label='RSSI(dB)', grid_lines='solid', orientation='vertical' )
x_axis  = bq.Axis(scale=xs, label='Dist(m)', grid_lines='solid')
tooltip = bq.Tooltip(fields=['x', 'y'], labels=['dist','rssi'])
points  = bq.Scatter(x=tlm.dist, y=tlm.rssi, scales={'x':xs,'y':ys}, default_size=20, labels=['data'], display_legend=True, tooltip=tooltip, unhovered_style={'opacity': 0.5})
line    = bq.Lines(x=np.sort(tlm.dist), y=tlm.rssi_dist, scales={'x': xs, 'y': ys}, colors=['red'], labels=['fit'], display_legend=True) 
fig     = bq.Figure(marks=[points, line], axes=[x_axis, y_axis], layout=Layout(width='30vw', height = '45vh'))
dist_max  = widgets.HTML(value="<b>Distance (m)</b> : "+tlm.max_dist+" / "+max_dist_tot)
speed_max = widgets.HTML(value="<b>Speed (km/h)</b> : "+tlm.max_speed+" / "+max_speed_tot)
alt_max   = widgets.HTML(value="<b>Altitude (m)</b> : "+tlm.max_alt+" / "+max_alt_tot)
trophee   = widgets.HTML('<center> <img src="images/trophy.jpg" align="middle" class="center"> </center>')
records   = VBox(children=[dist_max, speed_max, alt_max], layout=Layout(width='auto', height='20vh', align_items='center', border='solid 1px black'))
right     = VBox(children=[xy_var, fig, trophee, records], layout=Layout(flex='1 1 0%', width='auto', height='80vh', align_items='center', flex_direction="column"))

In [12]:
# INTERACTION

In [13]:
# function for updating all data : used withe slection and upload widgets
@out.capture()
def update_all(file):
    with out:
        tlm.get_data(file)
        x_var.options = tlm.data.columns
        y_var.options = tlm.data.columns
        x_var.value = 'Dist(m)'
        y_var.value = 'RSSI(dB)'
        tlm.data = tlm.data.sort_values(x_var.value)
        time_slider.max = len(tlm.time)
        # update map
        my_map.center   = np.mean(tlm.lat), np.mean(tlm.lon)
        my_map.layers[1].location = (tlm.lat.iloc[0], tlm.lon.iloc[0])
        my_map.layers[2].locations = [(x, y) for x, y in zip(tlm.lat, tlm.lon)]
        # update graph
        points.x = tlm.data[x_var.value]
        line.x   = tlm.data[x_var.value]
        points.y = tlm.data[y_var.value]
        line.y   = tlm.data[y_var.value]
        # update stats
        dist_max.value  = "<b>Distance (m)</b> : "+tlm.max_dist+" / "+max_dist_tot
        speed_max.value = "<b>Speed (km/h)</b> : "+tlm.max_speed+" / "+max_speed_tot
        alt_max.value   = "<b>Altitude (m)</b> : "+tlm.max_alt+" / "+max_alt_tot

In [14]:
@out.capture()
def file_change(change):
    with out:
        update_all('logs/'+change['new'])
file_select.observe(file_change, names='value')

@out.capture()
def upload_change(change):
    with out:
        uploaded_file = change['new']
        file = io.BytesIO(next(iter(uploaded_file.values()))['content'])
        update_all(file)
uploader.observe(upload_change, names='value')

@out.capture()
def xaxis_change(change):
    with out:
        if (change['new'] != 'Dist(m)') or (change['new'] != 'Time'):
            line.y   = []
        tlm.data = tlm.data.sort_values(change['new'])
        points.x = tlm.data[change['new']]
        points.y = tlm.data[y_var.value]
        x_axis.label = change['new']
        tooltip.labels[0] = change['new']
        #line.marker = 'circle'
        line.line_style = 'solid'
x_var.observe(xaxis_change, names='value')

@out.capture()
def yaxis_change(change):
    with out:
        points.y = tlm.data[change['new']]
        y_axis.label = change['new']
        tooltip.labels[1] = change['new']
y_var.observe(yaxis_change, names='value')

@out.capture()
def time_change(change):
    with out:
        last_time = change['new']
        my_map.layers[2].locations = [(x, y) for x, y in zip(tlm.lat[:last_time], tlm.lon[:last_time])]
        '''if graph_button.value == "Time":
            points.x, line.x = tlm.time[:last_time], tlm.time[:last_time]
            points.y, line.y = tlm.rssi[:last_time], tlm.rssi_time[:last_time]
        elif graph_button.value == "Distance":
            points.x = tlm.dist[:last_time]
            points.y = tlm.rssi[:last_time]'''
        points.x = tlm.data[x_var.value].iloc[:last_time]
        points.y = tlm.data[y_var.value].iloc[:last_time]
        dist_curr.value  = '<b>Distance (m)</b> : '+str(tlm.dist.iloc[last_time-1])
        speed_curr.value = '<b>Speed (km/h)</b> : '+str(tlm.speed.iloc[last_time-1])
        alt_curr.value   = '<b>Altitude (m)</b> : '+str(tlm.alt.iloc[last_time-1])
time_slider.observe(time_change, names='value')

In [15]:
# DISPLAY

In [16]:
display(HBox(children=[left, center, right]), out)

Output()

In [17]:
# 3D volume

In [18]:
#fig = ipv.figure(width=400, height=200,layout=Layout(width='14vw'))
#ipv.style.use('minimal')
#m = mesh.Mesh.from_file('stl/bixler.stl')
#ipv.plot_trisurf(m.x.flatten(), m.z.flatten(), m.y.flatten(), triangles=np.arange(m.x.shape[0] * 3).reshape((m.x.shape[0], 3)), color="gray")
#ipv.squarelim()

In [19]:
# video

In [20]:
#video = widgets.Video.from_file("videos/Big.Buck.Bunny.mp4", autoplay=False,layout=Layout(width='14vw'))

In [21]:
x_var = widgets.Dropdown(options=tlm.data.columns, description='X axis :', value = 'Dist(m)', layout=Layout(width='12vw'))
x_var

Dropdown(description='X axis :', index=17, layout=Layout(width='12vw'), options=('Date', 'Time', 'BtRx(V)', 'R…